# Задание 1

Есть таблица с действиями юзеров A/B-теста (task_1_events.csv):

events - установки приложения

userid - id юзера,

abgroup - группа A/B-теста,

ts - время совершения действия,

pdate - дата совершения действия.

Пользовательская сессия определяется по следующим правилам:

Новая сессия начинается после 30 минут бездействия.
Сессия прерывается при переходе между двумя датами.
Постройте таблицу с сессиями юзеров в формате:

user_id - id юзера

ab_group - группа A/B-теста,

start_ts - время старта сессии,

end_ts - время окончания сессии,

pdate - дата сессии.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Aviasales/task_1_events.csv')

In [4]:
df.head()

,user_id,ab_group,ts,pdate
0,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 12:24:33.553327,2023-05-02
1,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 12:32:23.215284,2023-05-02
2,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 12:33:01.050395,2023-05-02
3,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 12:36:18.233874,2023-05-02
4,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 12:36:45.210274,2023-05-02


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44568 entries, 0 to 44567
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   44568 non-null  object
 1   ab_group  44568 non-null  object
 2   ts        44568 non-null  object
 3   pdate     44568 non-null  object
dtypes: object(4)
memory usage: 1.4+ MB


In [6]:
df.describe()

,user_id,ab_group,ts,pdate
count,44568,44568,44568,44568
unique,5000,2,44568,8
top,rBMsVWPitiocbQAfe/TFAg==,A,2023-05-02 12:24:33.553327,2023-05-02
freq,196,22360,1,15239


In [7]:
# количество пользователей в группах
df.groupby('ab_group')['user_id'].nunique()

,user_id
ab_group,
A,2500
B,2500


In [8]:
#меняем тип данных в столбцах ts  на datetime
df['ts'] = pd.to_datetime(df['ts'])

In [9]:
# Определение временного диапазона эксперимента
experiment_start = df['ts'].min()
experiment_end = df['ts'].max()

print(f"Эксперимент начался: {experiment_start}")
print(f"Эксперимент закончился: {experiment_end}")

Эксперимент начался: 2023-05-01 00:01:33.241816
Эксперимент закончился: 2023-05-08 23:15:23.946260


In [10]:
# Определение количества дней эксперимента
experiment_duration = (experiment_end - experiment_start).days

print(f"Эксперимент длился {experiment_duration} дней.")

Эксперимент длился 7 дней.


In [11]:
#отсортировал по айди и времени, чтобы в дальнейшем определить является действие началом новой сессии
df = df.sort_values(by=['user_id', 'ts'])

In [12]:
#Времмено добавим 2 поля в таблицу, чтобы можно было определить разницу во времени между действиями
df['prev_ts'] = df.groupby('user_id')['ts'].shift(1)
df['prev_pdate'] = df.groupby('user_id')['pdate'].shift(1)

In [13]:
df.head()

,user_id,ab_group,ts,pdate,prev_ts,prev_pdate
22360,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:17:45.766570,2023-05-01,NaT,NaN
22361,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:23.403884,2023-05-01,2023-05-01 08:17:45.766570,2023-05-01
22362,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:24.197308,2023-05-01,2023-05-01 08:18:23.403884,2023-05-01
22363,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:26.551874,2023-05-01,2023-05-01 08:18:24.197308,2023-05-01
22364,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:18:28.284787,2023-05-01,2023-05-01 08:18:26.551874,2023-05-01


In [14]:
#меняем тип данных в столбцах ts и prev_ts на datetime
df['prev_ts'] = pd.to_datetime(df['prev_ts'])

In [15]:
#фильтрация(предыдущее время нан или даты не равны или разница во времени больше 30 мин)
df['new_session'] = ((df['prev_ts'].isna()) |  (df['pdate'] != df['prev_pdate']) |  ((df['ts'] - df['prev_ts']).dt.total_seconds() > 1800))

In [16]:
#создаем id каждой сессии
df['session_id'] = df['new_session'].cumsum()

In [17]:
#группируем данные по сессиям, агрегируем в каждой группе и сбрасываем индекс, чтобы получить та
df = df.groupby(['user_id', 'ab_group', 'session_id', 'pdate']).agg(
    start_ts=('ts', 'min'),
    end_ts=('ts', 'max')
    ).reset_index()

In [18]:
# убираем лишние столбцы и получаем новую таблицу
df = df[['user_id', 'ab_group', 'start_ts', 'end_ts', 'pdate']]

In [19]:
df.head(20)

,user_id,ab_group,start_ts,end_ts,pdate
0,2JM4L+sRXYIWAFSC8XaTPg==,B,2023-05-01 08:17:45.766570,2023-05-01 08:57:37.363424,2023-05-01
1,2OgK/ukRgvkWAOaa8XaTPg==,A,2023-05-02 12:24:33.553327,2023-05-02 12:37:57.761142,2023-05-02
2,3E69ZO0RJrEWACi28XaTPg==,A,2023-05-01 18:58:45.338597,2023-05-01 19:04:19.203838,2023-05-01
3,3E69ZO0RJrEWACi28XaTPg==,A,2023-05-04 10:01:39.700496,2023-05-04 10:07:39.858666,2023-05-04
4,3GsYmewRRswWABqG8XaTPg==,B,2023-05-03 05:00:41.797222,2023-05-03 05:18:12.137282,2023-05-03
5,4BNABuYRf+8WAKmUYvZgPg==,A,2023-05-03 07:31:46.831712,2023-05-03 07:39:39.146694,2023-05-03
6,9HKAHegRBmoWAAqW6VfbPg==,A,2023-05-01 07:19:36.837991,2023-05-01 07:35:11.687785,2023-05-01
7,CNlROesRiq0WAKmf8XaTPg==,A,2023-05-01 13:33:03.091067,2023-05-01 13:50:05.085239,2023-05-01
8,CkIBDFQ8sF2Iz2v5A1dcAg==,A,2023-05-01 12:34:18.732048,2023-05-01 12:39:56.669793,2023-05-01
9,CkIBDFQCusBsUkoQA0/DAg==,B,2023-05-01 04:23:26.682765,2023-05-01 04:51:59.406590,2023-05-01


# Задание 2

Был проведен A/B-тест.
В качестве данных используйте таблицу, построенную в предыдущем задании. Первая сессия юзера считается моментом попадания в A/B-тест.

Ключевая метрика эксперимента - конверсия во вторую сессию.
Сделайте вывод о том, какая группа выиграла в A/B-тесте. Ответ обоснуйте.

**A/B** **тест**



1.   Метрика эксперимента - конверсия во вторую сессию
2.   Анализ пользователей и определение параметров выборки не требуются, т.к.данные уже собраны
3.   Длительность эксперимента составляет 7 дней

     Группа А 2500 пользователей

     Группа В 2500 пользователей
4.   Т.к мы заранее не знаем, какая группа должна показать наибольшую конверсию, то выбирем двусторонний z-test.

     Н0: Конверсии во вторую сессию одинаковы для обеих групп cr_A = cr_B

     H1: Конверсии во вторую сессию различаются   cr_A != cr_B
5.   Данные распределены нормально, т.к. распределение пропорций приближается к нормальному при больших выборках(согласно ЦПТ)













In [20]:
# Добавляем номер сессии для каждого пользователя
df['session_number'] = df.groupby('user_id').cumcount() + 1

In [21]:
# Находим пользователей со второй сессией
second_session_users = df[df['session_number'] == 2]['user_id'].unique()

In [22]:
# Рассчитываем конверсию во вторую сессию
conversion = df[df['session_number'] == 1].groupby('ab_group')['user_id'].apply(
    lambda group: group.isin(second_session_users).mean()
).reset_index(name='conversion_rate')

In [23]:
print(conversion)

  ab_group  conversion_rate
0        A           0.1272
1        B           0.1412


In [24]:
!pip install statsmodels #install the statsmodels package since it's required
from statsmodels.stats.proportion import proportions_ztest

In [25]:
# Подготовка данных для теста
group_counts = df[df['session_number'] == 1].groupby('ab_group')['user_id'].nunique()
success_counts = df[df['session_number'] == 1].groupby('ab_group').apply(
    lambda g: g['user_id'].isin(second_session_users).sum()
)


<ipython-input-25-baeb34d37b5f>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  success_counts = df[df['session_number'] == 1].groupby('ab_group').apply(


In [26]:
# Z-тест
stat, p_value = proportions_ztest(count=success_counts, nobs=group_counts)

print(f"Z-statistic: {stat}, P-value: {p_value}")

Z-statistic: -1.4521049087396234, P-value: 0.14647243726715875


 p-valeu = 0.146, что значительно больше чем 0.05, значит у нас нет достаточно оснований, чтобы отклонить нулевую гипотезу. Разница в полученных конверсиях не является статистически значимой, поэтому нельзя сделать вывод какая группа выиграла в A/B тесте.